In [1]:
from dotenv import load_dotenv
import os

import logging
import re
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI # openai wrapper by langchain
from langchain.memory import ConversationBufferMemory #stoores the history of the conversation
from langchain_core.prompts import ChatPromptTemplate # defines a template for structuring prompts sent to the LLM
from langchain_core.runnables import RunnableSequence # combines multiple steps into a single executable chain
from langchain_core.prompts.chat import HumanMessagePromptTemplate, SystemMessagePromptTemplate #defines system and user part of the prompt
from tenacity import retry, stop_after_attempt, wait_fixed

from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail, Email, To, Content

In [3]:
logging.basicConfig(level = logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
load_dotenv(override=True)
groq_api_key = os.getenv("GROQ_API_KEY")
sendgrid_api_key = os.getenv("SENDGRID_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")

if not groq_api_key:
    logger.error("GROQ_API_KEY not found in environment variables.")
    print("Error: GROQ_API_KEY not found in environment variables.")
    exit(1)
if not sendgrid_api_key:
    logger.error("SENDGRID_API_KEY not found in environment variables.")
    print("Error: SENDGRID_API_KEY not found in environment variables.")
    exit(1)
if not langchain_api_key:
    logger.warning("LANGCHAIN_API_KEY not found; LangSmith tracing may not work.")

In [5]:
llm = ChatOpenAI(
    model="llama3-70b-8192",
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1",
)

In [6]:
instructions = {
    "agent1": """You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. 
Write a professional, serious cold email. Start with the salutation (e.g., 'Dear CEO') as specified in the input. Include a clear call-to-action (e.g., schedule a demo). Do not include a subject line, as it will be set separately. Do not include placeholders like [Your Name]. Return only the email body.""",
    "agent2": """You are a humorous, engaging sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. 
Write a witty, engaging cold email likely to get a response. Start with the salutation (e.g., 'Dear CEO') as specified in the input. Include a clear call-to-action (e.g., schedule a demo). Do not include a subject line, as it will be set separately. Do not include placeholders like [Your Name]. Return only the email body.""",
    "agent3": """You are a busy sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. 
Write a concise, to-the-point cold email. Start with the salutation (e.g., 'Dear CEO') as specified in the input. Include a clear call-to-action (e.g., schedule a demo). Do not include a subject line, as it will be set separately. Do not include placeholders like [Your Name]. Return only the email body.""",
    "evaluator": """You are an Evaluator Agent at ComplAI. Your goal is to select the best cold sales email from the provided drafts.
Follow these steps:
1. Review the three email drafts provided, separated by '---'.
2. Score each draft on:
   - Clarity (40%): Is the message clear, professional, and well-structured?
   - Engagement (40%): Does it capture attention with a compelling hook or tone?
   - Call-to-Action (20%): Does it include a strong, clear incentive to respond?
3. Select the draft with the highest weighted score.
4. Return only the raw content of the selected email draft (starting with the salutation, e.g., 'Dear CEO', with no additional text, explanations, or scoring details) or 'No suitable email found' if all drafts are invalid (e.g., contain errors or placeholders).""",
    "sales_manager": """You are a Sales Manager at ComplAI. Your goal is to coordinate the generation and sending of a cold sales email.
Follow these steps:
1. Use the provided drafts generated by sales agents.
2. Call the evaluator_tool to select the best draft.
3. If a suitable draft is found, call the send_email tool with the selected email content and recipient email.
4. If no suitable draft is found, return 'No suitable email found.'
Rules:
- Do not generate or modify drafts.
- Call the evaluator_tool exactly once.
- Call the send_email tool exactly once if a suitable draft is found.
Return only the result of the send_email tool or 'No suitable email found.'"""
}

In [8]:
memory_agent1 = ConversationBufferMemory(memory_key="professional_chat", k = 5, return_messages=True)
@tool
def agent1_tool(input: str) -> str:
    """Professional cold email agent for complAI"""
    logger.info(f"running agent1_tool with {input}")
    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(instructions["agent1"]),
        HumanMessagePromptTemplate.from_template("{input}")
    ])
    chain = RunnableSequence(prompt | llm)
    result = chain.invoke({"input": input, "chat_history":memory_agent1.buffer}).content
    logger.info(f"agent1_tool output: {result[:100]}...")
    return result

C:\Users\DeepADabhi\AppData\Local\Temp\ipykernel_17320\1852257344.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory_agent1 = ConversationBufferMemory(memory_key="professional_chat", k = 5, return_messages=True)


In [9]:
memory_agent2 = ConversationBufferMemory(memory_key = "humorous_chat_history", k=5, return_messages=True)
@tool
def agent2_tool(input: str) -> str:
    """Humoruous cold email agent for complAI"""
    logger.info(f"Running agent2 with input: {input}")
    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(instructions["agent2"]),
        HumanMessagePromptTemplate("{input}")
    ])
    chain = RunnableSequence(prompt | llm)
    result = chain.invoke({"input": input, "chat_history": memory_agent2.buffer}).content
    logger.info(f"agent2_tool output: {result[:100]}...")
    return result

In [10]:
memeory_agent3 = ConversationBufferMemory(memory_key="concise_chat_history", k = 5, return_messages=True)
@tool
def agent3_tool(input: str) -> str:
    """concise cold email agent for ComplAI"""
    logger.info(f"Running agent3_tool with input: {input}")
    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(instructions["agent3"]),
        HumanMessagePromptTemplate.from_template("{input}")
    ])
    chain = RunnableSequence(prompt|llm)
    result = chain.invoke({"input": input, "chat_history": memeory_agent3}).content
    logger.info(f"agent3_tool output: {result[:100]}...")
    return result

In [12]:
memory_evaluator = ConversationBufferMemory(memory_key = "evaluator_chat_history", k = 5, return_messages = True)
@tool
def evaluator_tool(input: str, drafts: str) -> str:
    """Evaluates email drafts and selects the best one based on clarity, engagement, and call-to-action."""
    logger.info(f"Running evaluator_tool with input: {input} and drafts: {drafts[:100]}...")
    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(instructions["evaluator"]),
        HumanMessagePromptTemplate.from_template("Input prompt: {input}\nDrafts:\n{drafts}\n\nReturn only the raw content of the selected email draft (starting with the salutation, e.g., 'Dear CEO') or 'No suitable email found'. Do not include any explanations, scoring details, or additional text.")
    ])
    chain = RunnableSequence(prompt | llm)
    result = chain.invoke({"input": input, "draft": drafts, "chat_history": memory_evaluator.buffer}).content
    logger.info(f"evaluator_tool output: {result[:100]}...")
    return result

In [13]:
@tool
def send_email(email_content: str, to_email: str) -> str:
    """Sends the selected email to the specified recipient using SendGrid."""
    logger.info(f"send_email called with content: {email_content[:50]}... to {to_email}")
    email_content = re.sub(r'^Subject:.*\n', '', email_content, flags=re.MULTILINE).strip()
    if not re.match(r'^[^\s]+@[^\s]+\.[^\s]+$', to_email):
        logger.error(f"Invalid recipient email: {to_email}")
        return f"Error: Invalid recipient email: {to_email}"
    
    try:
        message = Mail(
            from_email=Email("dabhideep44@gmail.com"),  
            to_emails=To(to_email),
            subject="Cold Sales Email from ComplAI",
            plain_text_content=Content("text/plain", email_content)
        )
        sg = SendGridAPIClient(sendgrid_api_key)
        response = sg.send(message)
        logger.info(f"Email sent successfully to {to_email}")
        return f"Email sent successfully to {to_email} : {email_content[:50]}..."
    except Exception as e:
        logger.error(f"Error sending email to {to_email}: {str(e)}")
        return f"Error sending email to {to_email}: {str(e)}"

In [ ]:
memory_sales_manager = ConversationBufferMemory(memory_key= "sales_manager_chat_history", k = 5, return_message=True)
@retry(srop=stop_after_attempt(3), wait=wait_fixed(2))
def generate_draft_with_retyr(tool, input_prompt):
    logger.info(f"Handling off to {tool.name} for draft generation")
    return tool.invoke(input_prompt)
def sales_manager(input)